In [91]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_validate, GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

In [92]:
df = pd.read_csv("SBAnational.csv")

/tmp/ipykernel_25157/3536806131.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("SBAnational.csv")


In [ ]:
df['MIS_Status'] = df.apply(
    lambda row: 'P I F' if pd.isna(row['ChgOffDate']) else 'CHGOFF' 
    if pd.isna(row['MIS_Status']) else row['MIS_Status'],
    axis=1
)
df = df.drop(["LoanNr_ChkDgt", "Name", "ChgOffDate", "DisbursementDate", "BalanceGross", "ChgOffPrinGr", "SBA_Appv"], axis=1)
df['ApprovalDate'] = pd.to_datetime(df['ApprovalDate'], format='%d-%b-%y')

df['ApprovalDate'] = df['ApprovalDate'].apply(
    lambda x: x - pd.DateOffset(years=100) if x.year > 2014 else x)

df['ApprovalFY'] = pd.to_numeric(df['ApprovalFY'], errors='coerce')
df['ApprovalFY'] = df['ApprovalFY'].astype('Int64')  # Int64 permet d'avoir des NaN

cols_to_convert = ['GrAppv', 'DisbursementGross']

for col in cols_to_convert:
    df[col] = df[col].astype(str)  # S'assure que c'est une string
    df[col] = df[col].str.replace('[\$,]', '', regex=True)  # Retire $ et ,
    df[col] = pd.to_numeric(df[col], errors='coerce')  # Convertit en float

df['State'] = df['State'].fillna(df['BankState'])
recession_start = pd.Timestamp('2007-12-01')
recession_end = pd.Timestamp('2009-06-30')
df['Recession'] = ((df['ApprovalDate'] >= recession_start) & (df['ApprovalDate'] <= recession_end)).astype(int)
# df['NewExist'] = df['NewExist'].map({2: 1, 1: 0, 0: np.nan})
# df['UrbanRural'] = df['UrbanRural'].map({2: 1, 1: 0, 0: np.nan})
df['HasFranchise'] = (df['FranchiseCode'] >= 100).astype(int)
df['RevLineCr'] = df['RevLineCr'].apply(lambda x: x if x in ['Y', 'N'] else np.nan)  # Garde seulement Y et N, sinon NaN
df = df[df['RevLineCr'].notnull()]
df['LowDoc'] = df['LowDoc'].apply(lambda x: x if x in ['Y', 'N'] else np.nan)  # Garde seulement Y et N, sinon NaN
df = df[df['LowDoc'].notnull()]
df["NAICS"] = df["NAICS"].astype(str).str[:2]
df['NAICS'] = df['NAICS'].replace('0', np.nan)
df = df[df['NAICS'].notnull()]
df = df[df['Bank'].notnull()]
df = df[df['State'].notnull()]
df = df.drop(['DisbursementGross','City','Zip','BankState','ApprovalDate','ApprovalFY','FranchiseCode'], axis=1)

In [101]:
features_of_interest = [
    'State', 'Bank', 'NAICS', 'Term', 'NoEmp', 'NewExist', 'CreateJob',
       'RetainedJob', 'UrbanRural', 'RevLineCr', 'LowDoc', 
       'GrAppv', 'Recession', 'HasFranchise'
    ]




data = df

target_name = "MIS_Status"
X, y = (
    data[features_of_interest],
    data[target_name]
)

In [102]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.1, random_state=42, stratify=y)

In [103]:
X['RevLineCr']

0         N
1         N
2         N
5         N
7         N
         ..
899145    Y
899146    N
899157    N
899160    Y
899161    N
Name: RevLineCr, Length: 461019, dtype: object

In [104]:
from catboost import CatBoostClassifier

cat_features = ['NAICS', 'Bank', 'State', 'RevLineCr', 'LowDoc']

# Initialisation du modèle avec AUC comme métrique
model = CatBoostClassifier(
    iterations=500, 
    learning_rate=0.1, 
    depth=6, 
    cat_features=cat_features, 
    eval_metric="AUC",  # Utilisation de l'AUC
    verbose=100
)

# Entraînement
model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=50, verbose=100)

# Prédiction des probabilités pour calculer l'AUC
y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probabilité de la classe positive

# Calcul de l'AUC
auc_score = roc_auc_score(y_test, y_pred_proba)
print(f"AUC: {auc_score:.4f}")


0:	test: 0.9022034	best: 0.9022034 (0)	total: 220ms	remaining: 1m 49s
100:	test: 0.9758425	best: 0.9758425 (100)	total: 16.8s	remaining: 1m 6s
200:	test: 0.9798300	best: 0.9798300 (200)	total: 33.7s	remaining: 50.1s
300:	test: 0.9814191	best: 0.9814191 (300)	total: 50.7s	remaining: 33.5s
400:	test: 0.9822350	best: 0.9822369 (399)	total: 1m 7s	remaining: 16.7s
499:	test: 0.9827398	best: 0.9827398 (499)	total: 1m 24s	remaining: 0us

bestTest = 0.9827398247
bestIteration = 499

AUC: 0.9827


In [105]:
from catboost import CatBoostClassifier

cat_features = ['NAICS', 'Bank', 'State', 'RevLineCr', 'LowDoc']

# Initialisation du modèle avec AUC comme métrique
model = CatBoostClassifier(
    iterations=500, 
    learning_rate=0.1, 
    depth=6, 
    cat_features=cat_features, 
    eval_metric='Precision',  # Utilisation de l'AUC
    verbose=100
)

# Entraînement
model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=50, verbose=100)

precision = model.score(X_test, y_test)
print(f'Précision sur l\'ensemble de test : {precision:.4f}')

0:	learn: 0.9442760	test: 0.9446696	best: 0.9446696 (0)	total: 201ms	remaining: 1m 40s
100:	learn: 0.9566834	test: 0.9563334	best: 0.9563334 (100)	total: 16.8s	remaining: 1m 6s
200:	learn: 0.9620437	test: 0.9614662	best: 0.9614662 (200)	total: 32.3s	remaining: 48s
300:	learn: 0.9645826	test: 0.9642194	best: 0.9642194 (299)	total: 47.9s	remaining: 31.6s
400:	learn: 0.9659516	test: 0.9655474	best: 0.9655474 (400)	total: 1m 3s	remaining: 15.8s
499:	learn: 0.9671432	test: 0.9663558	best: 0.9663558 (499)	total: 1m 21s	remaining: 0us

bestTest = 0.9663557537
bestIteration = 499

Précision sur l'ensemble de test : 0.9538
